# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [4]:
# TODO: Write a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
act as a QA engineer, using the app_code as context, generate a pytest test function for the POST /users/ endpoint,
asserting that a user is created successfully (e.g., checking for a 201 created or 200 OK status code and verifying the
response body) generate another test for the GET /users/ endpoint save the genereated tests into a file named tests/test_main_simple.py
use {app_code} as context for the tests
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py")
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
# tests/test_main_simple.py

import pytest
from fastapi.testclient import TestClient
from main import app, Base, engine, SessionLocal
from sqlalchemy.orm import sessionmaker

# Initialize TestClient
client = TestClient(app)

# Create a new database session for testing
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Drop and recreate the tables for a fresh start before each test
@pytest.fixture(autouse=True)
def setup_and_teardown_db():
    Base.metadata.drop_all(bind=engine)
    Base.metadata.create_all(bind=engine)
    yield
    Base.metadata.drop_all(bind=engine)

def test_create_user():
    # Define the user data to send in the request
    user_data = {
        "username": "testuser",
        "email": "testuser@example.com",
        "full_name": "Test User",
        "hashed_password": "hashedpassword"
    }
    
    # Send a POST request to create a new user
    response = client.post("/users", json=user_data)

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [5]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
use the {app_code} as context, write two new test functions: a test for the POST /users/ endpoint that tries
to create a user with an emails that already exists, asserting that the API returns a 400 Bad Request error,
and another test for GET /users/user_id andpoint that requests a non-existent user ID, asserting that the API
returns a 404 Not Found error
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
Next, create a test file, say `test_main.py`, and add the following test functions:


### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [6]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f"""
generate a pytest fixture, it should configure a temporary, in-memory SQLite database using SQLAlchemy,
it needs to create all the database tables before the test runs and tear them down afterward, crucially,
it must override the get_db dependency in your FastAPT app to use this temporary database during tests, save the
generated fixture code to a special file named tests/conftest.py
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py")
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
finally, create a new test file tests/test_main_with_fixture.py
and rewrite the happy-path tests from Challenge 1 to use the new database fixture
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py")
else:
    print("Skipping test refactoring because app context is missing.")

--- Generating Pytest DB Fixture ---
import pytest
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
from fastapi.testclient import TestClient
from your_app_name.main import app  # Import your FastAPI app
from your_app_name.database import Base, get_db  # Import your Base and get_db

# Create a new SQLAlchemy engine instance
SQLALCHEMY_DATABASE_URL = "sqlite:///:memory:"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})

# Create a configured "Session" class
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create a new base class for declarative class definitions
Base = declarative_base()

# Define your models here or import them
# Example:
# class User(Base):
#     __tablename__ = 'users'
#     id = Column(Integer, primary_key=True, index=True)
#     name = Column(String, index=True)

@pytest.fixture(scope="function")
def db_session():
    # Create the database ta

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.